# Support vector machine
Tentativo di fare la previsione con una support vector machine

In [54]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
#display(df)
#separating y from x
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

In [55]:
#declaring classifier for 'one versus one' case
clf = svm.SVC(decision_function_shape='ovo')
print(type(clf))

<class 'sklearn.svm._classes.SVC'>


In [56]:
#fitting classifier
clf.fit(X, y)

SVC(decision_function_shape='ovo')

In [57]:
#test_res = clf.decision_function(X[100,:])

In [58]:
#Possiamo vedere i support vector se vogliamo
clf.support_vectors_

array([[1.0000e+00, 7.2694e-01, 1.4742e+00, ..., 5.2323e-03, 2.7477e-04,
        1.1756e+00],
       [2.0000e+00, 7.4173e-01, 1.5257e+00, ..., 2.7080e-03, 7.4846e-05,
        6.9659e-01],
       [3.0000e+00, 7.6722e-01, 1.5725e+00, ..., 9.2068e-04, 3.7886e-05,
        4.4348e-01],
       ...,
       [8.0000e+00, 3.5344e-01, 1.0329e+00, ..., 1.6123e-02, 4.5288e-04,
        1.6935e+00],
       [9.0000e+00, 5.9988e-01, 1.1427e+00, ..., 6.1900e-03, 2.6454e-04,
        1.1526e+00],
       [1.0000e+01, 4.7195e-01, 1.0901e+00, ..., 1.3487e-02, 3.2855e-04,
        1.5623e+00]])

## Static test

In [132]:
#dividing the dataset for a static test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:16], df.iloc[:, 0], test_size=0.3) # 70% training and 30% test

Nota: se vuoi ottenere sempre lo stesso split (ad esempio per fare delle prove), aggiungi il parametro "random_state = num" dove num è un intero costante

In [153]:
stat_clf = svm.SVC(kernel='linear')    #remember that the default kernel is rbf. 
#Try changing kernel between: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} 
stat_clf.fit(X_train, y_train)

SVC(kernel='linear')

<class 'sklearn.svm._classes.SVC'>
<class 'sklearn.svm._classes.SVC'>


### Nota
Vediamo cosa succede se salviamo ciò che ritorna il metodo "fit" in una variabile

In [156]:
what_is_this = stat_clf.fit(X_train, y_train) #stessi parametri di stat_clf nel blocco precedente
print(type(stat_clf))
print(type(what_is_this))

<class 'sklearn.svm._classes.SVC'>
<class 'sklearn.svm._classes.SVC'>


In [157]:
#let's make a prediction on the test data
y_pred = stat_clf.predict(X_test)
y_pred2 = what_is_this.predict(X_test)
y_pred - y_pred2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Secondo me
stat_clf e what_is_this sono probabilmente lo stesso classifier SVM, perchè hanno sia lo stesso "type" che le stesse previsioni nel test set.

### Valutiamo il nostro modello statico

In [159]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3431372549019608


Valore abbastanza basso. Mi sembra che i risultati migliori si ottengano con il kernel lineare.

## Cross validation
Facciamo learning tramite cross validation su un terzo modello "clf_cv"

In [177]:
#print(metrics.get_scorer_names())

In [190]:
clf_cv = svm.SVC(decision_function_shape='ovo')
k = 5
effect_cv = cross_validate(clf_cv, X, y, cv=k, scoring=('accuracy'))
effect_cv
print("Mean test_score:", mean(effect_cv['test_score']))

{'fit_time': array([0.01212358, 0.01192689, 0.00936079, 0.01250625, 0.01175809]),
 'score_time': array([0.00372338, 0.00360727, 0.00377464, 0.00373173, 0.00292754]),
 'test_score': array([0.11764706, 0.19117647, 0.14705882, 0.17647059, 0.19117647])}

Mean test_score: 0.16470588235294117
